In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


from xgboost import XGBClassifier
from xgboost import plot_importance

import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
df = pd.read_csv('../data/raw_postpro.csv', encoding='cp949')
df1 = df[['Machinery', 'Assembly' , "Part No.1","청구품목", 'key2']]

In [41]:
from sklearn.preprocessing import LabelEncoder

# 기존 데이터프레임의 라벨 인코딩 방식 확인
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ['Machinery', 'Assembly' , "Part No.1","청구품목", 'key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = LabelEncoder()
    le.fit(df1[column])
    label_encoders[column] = le
    df1[column+"_encoded"] = le.transform(df1[column])

C:\Users\SW\AppData\Local\Temp\ipykernel_9752\2691750297.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[column+"_encoded"] = le.transform(df1[column])
C:\Users\SW\AppData\Local\Temp\ipykernel_9752\2691750297.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[column+"_encoded"] = le.transform(df1[column])
C:\Users\SW\AppData\Local\Temp\ipykernel_9752\2691750297.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [42]:
df.columns

Index(['청구서번호', 'No.', 'Subject', 'Machinery', 'Assembly', '청구품목', 'Part No.1',
       'Part No.2', 'key1', 'key2', '리드타임_음수제거', '청구량', '견적', '견적수량', '견적화폐',
       '견적단가', '발주번호', '발주처', '발주', '발주수량', '발주금액', 'D/T', '미입고 기간', '리드타임',
       '창고입고', '창고입고수량', 'Control No.', '입고창고', '창고출고', '창고출고수량', '출고선박',
       '출고운반선', '선박입고', '선박입고수량', '완료 여부'],
      dtype='object')

In [43]:
df1

,Machinery,Assembly,Part No.1,청구품목,key2,Machinery_encoded,Assembly_encoded,Part No.1_encoded,청구품목_encoded,key2_encoded
0,NO.1 GENERATOR ENGINE,323-6480 LINES GP-FUEL,7.00E-275,SEAL-O-RING-STOR,COOLER,108,281,5759,4848,18
1,NO.2 GENERATOR ENGINE,GASKET KIT,7.00E-275,OIL COOLER & LINES,COOLER,113,1180,5759,3692,18
2,NO.2 GENERATOR ENGINE,285-8374 MANIFOLD GP-EXH,7.00E-275,WASHER,COOLER,113,206,5759,5915,18
3,NO.1 GENERATOR ENGINE,159-8828 TURBO GP,7.00E-275,BOLT-HIGH TEMP,COOLER,108,69,5759,696,18
4,NO.1 GENERATOR ENGINE,2N4727 INSTRUMNT PANEL GP,7.00E-275,SEAL,COOLER,108,237,5759,4681,18
...,...,...,...,...,...,...,...,...,...,...
20512,HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,φ150 × φ85 × 695st,HYD. CYLINDER_NET BOAT DAVIT,CYLINDER,63,471,8126,2811,20
20513,HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,φ160 × φ112 × 1130st,HYD. CYLINDER_N.D CRANE_BOOM_LIFTER,CYLINDER,63,471,8127,2810,20
20514,6M NET BOAT,ELECTRIC CLUTCH,Φ305X5V-3,PULLY,BELT,0,900,8123,4151,5
20515,7M NET BOAT,HAND HYDRAULIC STEERING SYSTEM,Φ50 x S195 CYLINDER,CYLINDER,CYLINDER,4,1210,8124,1592,20


In [44]:
# df1
X = df1[['Machinery_encoded', 'Assembly_encoded', "Part No.1_encoded", '청구품목_encoded']]
y = df1["key2_encoded"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.3)
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# xgboost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=61)

# 모델 학습
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=61,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [65]:
X_train

,Machinery_encoded,Assembly_encoded,Part No.1_encoded,청구품목_encoded
13153,92,898,4433,5217
1598,149,1164,4006,831
15313,39,1573,5305,5465
1546,27,1390,3800,4788
16043,116,1180,5740,2088
...,...,...,...,...
11532,73,1648,3466,5502
16065,61,245,5680,4843
14501,116,309,5034,4840
14555,108,61,5050,5142


In [45]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = model.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

F1-score: 0.8393613937690496


In [53]:
label_encoders.keys()

dict_keys(['Machinery', 'Assembly', 'Part No.1', '청구품목', 'key2'])

In [52]:
new_data = np.array([['SKIFF BOAT', 'IMPLANTATION INSTRUMENTS', '14.9795', 'EXHAUST GAS TEMPERATURE INDICATOR 0 TO 700℃']])
col_name = [['Machinery', 'Assembly' , "Part No.1","청구품목", 'key2']]

idx = 0
for a,b in zip(new_data, col_name):
    # temp = []
    # df.apply(le.fit_transform)
    # temp.append(label_encoders[b].fit(a))
    b[idx] = label_encoders[b].fit_transform(a)
    idx = idx +1
    print(a)
    print(b)
# temp

TypeError: unhashable type: 'list'

In [54]:
from sklearn.preprocessing import LabelEncoder

# 주어진 데이터
data = ['SKIFF BOAT', 'IMPLANTATION INSTRUMENTS', '14.9795', 'EXHAUST GAS TEMPERATURE INDICATOR 0 TO 700℃']

# 데이터 라벨링
encoded_data = []
for i in range(len(data)):
    label = list(label_encoders.keys())[i]  # 라벨링할 열의 키
    encoder = label_encoders[label]  # 해당 열의 LabelEncoder 객체
    encoded_value = encoder.transform([data[i]])[0]  # 데이터 라벨링
    encoded_data.append(encoded_value)

print(encoded_data)


[148, 1248, 1703, 1887]


In [66]:
encoded_data = np.array(encoded_data)
encoded_data = encoded_data.reshape((1,4))
encoded_data

array([[ 148, 1248, 1703, 1887]])

In [67]:
model.predict(encoded_data)

array([25])

In [69]:
## decoded_data 디코딩해야함

In [68]:
decoded_data = label_encoders["key2"].inverse_transform(encoded_data)

ValueError: y should be a 1d array, got an array of shape (1, 4) instead.